In [1]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Dataset
!ls

/content/drive/MyDrive/Dataset
a1_RestaurantReviews_HistoricDump.tsv  c1_BoW_Sentiment_Model.pkl
a2_RestaurantReviews_FreshDump.tsv     c2_Classifier_Sentiment_Model


In [7]:
dataset = pd.read_csv('a2_RestaurantReviews_FreshDump.tsv', delimiter = '\t', quoting = 3)

In [8]:
dataset.head

<bound method NDFrame.head of                                                Review
0                         Spend your money elsewhere.
1   Their regular toasted bread was equally satisf...
2   The Buffet at Bellagio was far from what I ant...
3                    And the drinks are WEAK, people!
4                          -My order was not correct.
..                                                ...
95  I think food should have flavor and texture an...
96                           Appetite instantly gone.
97  Overall I was not impressed and would not go b...
98  The whole experience was underwhelming, and I ...
99  Then, as if I hadn't wasted enough of my life ...

[100 rows x 1 columns]>

In [10]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
corpus=[]

for i in range(0, 100):
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [12]:
# Loading BoW dictionary
from sklearn.feature_extraction.text import CountVectorizer
import pickle
cvFile='c1_BoW_Sentiment_Model.pkl'
cv = pickle.load(open(cvFile, "rb"))

In [13]:
X_fresh = cv.transform(corpus).toarray()
X_fresh.shape

(100, 1420)

In [14]:
import joblib
classifier = joblib.load('c2_Classifier_Sentiment_Model')

In [15]:
y_pred = classifier.predict(X_fresh)
print(y_pred)

[0 1 1 0 0 1 0 0 0 1 1 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0
 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0]


In [16]:
dataset['predicted_label'] = y_pred.tolist()
dataset.head()

,Review,predicted_label
0,Spend your money elsewhere.,0
1,Their regular toasted bread was equally satisf...,1
2,The Buffet at Bellagio was far from what I ant...,1
3,"And the drinks are WEAK, people!",0
4,-My order was not correct.,0


In [17]:
dataset.to_csv("c3_Predicted_Sentiments_Fresh_Dump.tsv", sep='\t', encoding='UTF-8', index=False)